In [1]:
!pip install pandas > None
!pip install sentence-transformers > None
!pip install tqdm > None
!pip install numpy > None


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import os.path as path
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

In [ ]:
root_dir = path.dirname(path.abspath(os.getcwd()))
data_dir = path.join(root_dir, 'data')
processed_data_dir = path.join(data_dir, 'processed')
embeddings_dir = path.join(data_dir, 'embeddings')

os.makedirs(embeddings_dir, exist_ok=True)

processed_vacancies_path = path.join(processed_data_dir, 'vacancies.csv')
processed_cvs_path = path.join(processed_data_dir, 'cvs.csv')

vacancies_embeddings_path = path.join(embeddings_dir, 'vacancies.csv')
cvs_embeddings_path = path.join(embeddings_dir, 'cvs.csv')

processed_cvs_path

In [ ]:
processed_vacancies = pd.read_csv(processed_vacancies_path)
processed_vacancies

In [ ]:
processed_cvs = pd.read_csv(processed_cvs_path)
processed_cvs

In [8]:
model = SentenceTransformer('cointegrated/LaBSE-en-ru')

In [9]:
n_batch = 1000

def chunks(n, lst):
    for i in range(0, len(lst), n):
        print(i)
        yield lst[i:i + n]

def to_csv(filename, df):
    print(f"Total texts found: {len(df)}")
    with open(filename, 'w') as file:
        file.write('index,embeddings\n')  # Заголовок для столбцов
        for i, chunk in enumerate(chunks(n_batch, df[['index', 'text']].values.tolist())):
            try:
                indices, texts = zip(*chunk)
                encoded_chunk = model.encode(texts)
                content = '\n'.join([f'{index},{":".join(map(str, emb))}' for index, emb in zip(indices, encoded_chunk)])
                file.write(content + '\n')
            except BaseException as e:
                print(f"Error in {i} chunk: {e}")

In [ ]:
to_csv(vacancies_embeddings_path, processed_vacancies[['index', 'text']])

Total texts found: 96225
0
1000
2000
3000


In [ ]:
to_csv(cvs_embeddings_path, processed_cvs[['index', 'text']])